In [1]:
import os
import pandas as pd
import re

def extract_info_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 提取标题和内容
    title_match = re.search(r'^(.*?)\n', content)
    title = title_match.group(1) if title_match else ''
    
    content_match = re.search(r'\n\n(.*)', content, re.DOTALL)
    content = content_match.group(1) if content_match else ''
    
    # 提取国家、城市和年份
    country_match = re.search(r'^(.*?)的', title)
    country = country_match.group(1) if country_match else ''
    
    city_match = re.search(r'位于(.*?)，', content)
    city = city_match.group(1) if city_match else ''
    
    year_match = re.search(r'建于公元(\d+)年', content)
    year = year_match.group(1) if year_match else ''
    if year:
        year = int(year)
        if '公元前' in content:
            year = -year
    
    return {
        'title': title,
        'content': content,
        'country': country,
        'city': city,
        'year': year
    }

def generate_landmarks_csv(input_dir, output_file):
    data = []
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_dir, filename)
            info = extract_info_from_file(file_path)
            data.append(info)
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"已生成 {output_file}，包含 {len(df)} 条记录。")

# 生成CSV文件
generate_landmarks_csv('temp_docs', 'landmarks_data.csv')

# 显示生成的CSV文件的前几行
df = pd.read_csv('landmarks_data.csv')
print(df.head())

已生成 landmarks_data.csv，包含 100 条记录。
                    title                                            content  \
0            Yemen的美丽的纪念碑  Yemen的美丽的纪念碑位于Melindabury，是Yemen最著名的纪念碑之一。它建于公...   
1           Guinea的迷人的博物馆  Guinea的迷人的博物馆位于Lucasfort，是Guinea最著名的博物馆之一。它建于公...   
2  Papua New Guinea的迷人的公园  Papua New Guinea的迷人的公园位于Meganview，是Papua New G...   
3     Tunisia的令人印象深刻的自然景观  Tunisia的令人印象深刻的自然景观位于Sandrachester，是Tunisia最著名...   
4          Bolivia的壮观的纪念碑  Bolivia的壮观的纪念碑位于Estradaview，是Bolivia最著名的纪念碑之一。...   

            country           city    year  
0             Yemen    Melindabury     NaN  
1            Guinea      Lucasfort     NaN  
2  Papua New Guinea      Meganview     NaN  
3           Tunisia  Sandrachester     NaN  
4           Bolivia    Estradaview  1382.0  


In [2]:
import networkx as nx
import pandas as pd
from pyvis.network import Network

def create_knowledge_graph(data):
    G = nx.Graph()
    for _, row in data.iterrows():
        title = row['title']
        content = row['content']
        # 提取国家和城市信息
        country = title.split('的')[0]
        city = content.split('位于')[1].split('，')[0]
        
        # 添加节点
        G.add_node(title, type='landmark')
        G.add_node(country, type='country')
        G.add_node(city, type='city')
        
        # 添加边
        G.add_edge(title, country, relation='located_in')
        G.add_edge(title, city, relation='located_in')
        G.add_edge(city, country, relation='part_of')
    
    return G

# 读取之前生成的数据
df = pd.read_csv('landmarks_data.csv')  # 假设我们之前将生成的数据保存为CSV文件

# 创建知识图谱
kg = create_knowledge_graph(df)

print(f"知识图谱创建完成。节点数: {kg.number_of_nodes()}, 边数: {kg.number_of_edges()}")

# 可视化知识图谱（可选）
def visualize_graph(G, filename='knowledge_graph.html'):
    net = Network(notebook=True, width="100%", height="500px")
    for node in G.nodes():
        net.add_node(node, label=node, title=G.nodes[node]['type'])
    for edge in G.edges():
        net.add_edge(edge[0], edge[1], title=G[edge[0]][edge[1]]['relation'])
    net.save_graph(filename)
    print(f"知识图谱已保存为 {filename}")

visualize_graph(kg)

知识图谱创建完成。节点数: 284, 边数: 300
知识图谱已保存为 knowledge_graph.html


In [3]:
from llama_index.core import Document
from llama_index.core.retrievers import BaseRetriever

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


class KGEnhancedRetriever(BaseRetriever):
    def __init__(self, vector_retriever, knowledge_graph):
        self.vector_retriever = vector_retriever
        self.kg = knowledge_graph

    def _retrieve(self, query, **kwargs):
        # 首先使用向量检索器获取初始结果
        initial_results = self.vector_retriever._retrieve(query, **kwargs)

        enhanced_results = []
        for node in initial_results:
            # 获取与检索结果相关的知识图谱信息
            title = node.metadata.get('title', '')
            if title in self.kg:
                neighbors = list(self.kg.neighbors(title))
                kg_info = f"Related information: {title} is connected to {', '.join(neighbors)}."
                
                # 创建一个新的Document对象，包含原始内容和知识图谱信息
                enhanced_content = f"{node.text}\n\n{kg_info}"
                enhanced_node = Document(text=enhanced_content, metadata=node.metadata)
                enhanced_results.append(enhanced_node)
            else:
                enhanced_results.append(node)

        return enhanced_results

# 创建增强的检索器
documents = SimpleDirectoryReader('temp_docs').load_data()

# 初始化嵌入模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)

# 创建向量索引
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
kg_enhanced_retriever = KGEnhancedRetriever(vector_index.as_retriever(similarity_top_k=3), kg)

/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.


In [ ]:
import networkx as nx
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


def kg_based_reasoning(query, kg, initial_entities):
    def find_path(start, end):
        try:
            path = nx.shortest_path(kg, start, end)
            return ' -> '.join(path)
        except nx.NetworkXNoPath:
            return None

    reasoning_results = []
    for entity in initial_entities:
        neighbors = list(kg.neighbors(entity))
        reasoning_results.append(f"Entity: {entity}")
        reasoning_results.append(f"Connected to: {', '.join(neighbors)}")
        
        # 寻找与查询相关的路径
        for neighbor in neighbors:
            path = find_path(entity, neighbor)
            if path:
                reasoning_results.append(f"Path from {entity} to {neighbor}: {path}")

    return "\n".join(reasoning_results)


# 更新查询引擎以包含知识图谱推理
def get_kg_enhanced_query_engine(llm, prompt, kg_enhanced_retriever, kg):
    def postprocess_response(response, query, kg):
        initial_entities = [node.metadata.get('title', '') for node in response.source_nodes]
        kg_reasoning = kg_based_reasoning(query, kg, initial_entities)
        return f"{response.response}\n\nAdditional insights from knowledge graph:\n{kg_reasoning}"

    return RetrieverQueryEngine.from_args(
        kg_enhanced_retriever,
        llm=llm,
        text_qa_template=prompt,
        node_postprocessors=[
            SimilarityPostprocessor(similarity_cutoff=0.7),
            lambda x: postprocess_response(x, x.query, kg)
        ]
    )

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM


# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt


def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


def get_llm(model_name):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=model_name,
        context_window=3900,
        max_new_tokens=256,
        model_kwargs={"trust_remote_code": True},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="auto",
    )
    return llm

# 初始化三个LLM
base_path = "/home/work/var/data/ssr-share-data/"
# qwen_14b = get_llm(base_path + "Qwen-14B-Chat")
chatglm3 = get_llm(base_path + "chatglm3-6b")
qwen_7b = get_llm(base_path + "Qwen1.5-7B-Chat")

llm_dict = {
    # "Qwen-14B": qwen_14b,
    "ChatGLM3-6B": chatglm3,
    "Qwen1.5-7B": qwen_7b
}


In [ ]:
from llama_index.core import PromptTemplate

# 基础提示模板
base_prompt = PromptTemplate(
    "请根据以下信息回答问题。如果无法从给定信息中找到答案，请说 '我没有足够的信息来回答这个问题'。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n回答: "
)

# 思维链提示模板
cot_prompt = PromptTemplate(
    "请根据以下信息回答问题。在给出最终答案之前，请先逐步分析你的思考过程。如果无法从给定信息中找到答案，请解释为什么。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n思考过程:\n1."
)

# 多角度提示模板
multi_perspective_prompt = PromptTemplate(
    "请根据以下信息从多个角度回答问题。考虑不同的观点和可能性。如果信息不足，请指出并解释为什么。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n多角度分析:\n1."
)

prompts = {
    "基础": base_prompt,
    "思维链": cot_prompt,
    "多角度": multi_perspective_prompt
}

In [ ]:
import time

# 使用之前定义的测试问题
test_questions = [
    "哪些国家有著名的自然景观？",
    "有哪些建于公元前的历史遗迹？",
    "世界上最高的现代建筑是什么？",
    "哪些宗教场所最受游客欢迎？",
    "有哪些著名的桥梁地标？"
]

results = {}

for llm_name, llm in llm_dict.items():
    for prompt_name, prompt in prompts.items():
        key = f"{llm_name}-{prompt_name}-KG"
        query_engine = get_kg_enhanced_query_engine(llm, prompt, kg_enhanced_retriever, kg)
        
        start_time = time.time()
        responses = []
        for question in test_questions:
            response = query_engine.query(question)
            responses.append(str(response))
        end_time = time.time()
        
        results[key] = {
            "responses": responses,
            "time": end_time - start_time
        }

# 打印结果
for key, result in results.items():
    print(f"Model-Prompt-KG: {key}")
    print(f"Total time: {result['time']:.2f} seconds")
    for i, response in enumerate(result['responses']):
        print(f"Q{i+1}: {test_questions[i]}")
        print(f"A{i+1}: {response[:300]}...")  # 打印前300个字符
    print("-" * 50)

In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from typing import List
import networkx as nx
import pandas as pd
import time

# 假设我们已经有了知识图谱 kg 和之前定义的 llm_dict 和 prompts

class KGEnhancedRetriever(BaseRetriever):
    def __init__(self, vector_retriever, knowledge_graph):
        self.vector_retriever = vector_retriever
        self.kg = knowledge_graph

    def _retrieve(self, query, **kwargs):
        initial_results = self.vector_retriever.retrieve(query, **kwargs)
        enhanced_results = []
        for node in initial_results:
            title = node.metadata.get('title', '')
            if title in self.kg:
                neighbors = list(self.kg.neighbors(title))
                kg_info = f"Related information: {title} is connected to {', '.join(neighbors)}."
                enhanced_content = f"{node.text}\n\n{kg_info}"
                enhanced_node = Document(text=enhanced_content, metadata=node.metadata)
                enhanced_results.append(enhanced_node)
            else:
                enhanced_results.append(node)
        return enhanced_results

def kg_based_reasoning(query, kg, initial_entities):
    def find_path(start, end):
        try:
            path = nx.shortest_path(kg, start, end)
            return ' -> '.join(path)
        except nx.NetworkXNoPath:
            return None

    reasoning_results = []
    for entity in initial_entities:
        neighbors = list(kg.neighbors(entity))
        reasoning_results.append(f"Entity: {entity}")
        reasoning_results.append(f"Connected to: {', '.join(neighbors)}")
        
        for neighbor in neighbors:
            path = find_path(entity, neighbor)
            if path:
                reasoning_results.append(f"Path from {entity} to {neighbor}: {path}")

    return "\n".join(reasoning_results)

def get_kg_enhanced_query_engine(llm, prompt, kg_enhanced_retriever, kg):
    def postprocess_response(response, query):
        initial_entities = [node.metadata.get('title', '') for node in response.source_nodes]
        kg_reasoning = kg_based_reasoning(query, kg, initial_entities)
        return f"{response.response}\n\nAdditional insights from knowledge graph:\n{kg_reasoning}"

    return RetrieverQueryEngine.from_args(
        retriever=kg_enhanced_retriever,
        llm=llm,
        text_qa_template=prompt,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
        response_mode="compact",
        # 在这里使用一个回调函数来后处理结果
        callback_manager=None,
        use_async=False,
        streaming=False,
        # 添加一个自定义的后处理步骤
        custom_postprocessors=[
            lambda x: postprocess_response(x, x.query_str)
        ]
    )

# 假设我们已经有了向量索引 vector_index
kg_enhanced_retriever = KGEnhancedRetriever(vector_index.as_retriever(similarity_top_k=3), kg)

# 测试问题
test_questions = [
    "哪些国家有著名的自然景观？",
    "有哪些建于公元前的历史遗迹？",
    "世界上最高的现代建筑是什么？",
    "哪些宗教场所最受游客欢迎？",
    "有哪些著名的桥梁地标？"
]

results = {}

for llm_name, llm in llm_dict.items():
    for prompt_name, prompt in prompts.items():
        key = f"{llm_name}-{prompt_name}-KG"
        query_engine = get_kg_enhanced_query_engine(llm, prompt, kg_enhanced_retriever, kg)
        
        start_time = time.time()
        responses = []
        for question in test_questions:
            response = query_engine.query(question)
            responses.append(str(response))
        end_time = time.time()
        
        results[key] = {
            "responses": responses,
            "time": end_time - start_time
        }

# 打印结果
for key, result in results.items():
    print(f"Model-Prompt-KG: {key}")
    print(f"Total time: {result['time']:.2f} seconds")
    for i, response in enumerate(result['responses']):
        print(f"Q{i+1}: {test_questions[i]}")
        print(f"A{i+1}: {response[:300]}...")  # 打印前300个字符
    print("-" * 50)

/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model-Prompt-KG: ChatGLM3-6B-基础-KG
Total time: 14.70 seconds
Q1: 哪些国家有著名的自然景观？
A1: 以下国家有著名的自然景观：

1. Puerto Rico
2. Botswana
3. Tunisia...
Q2: 有哪些建于公元前的历史遗迹？
A2: 建于公元前的历史遗迹包括：

1. Somalia的宏伟的纪念碑，建于公元-2394年（公元前）
2. Suriname的美丽的历史遗迹，建于公元-1358年（公元前）
3. Germany的令人印象深刻的历史遗迹，建于公元-1477年（公元前）...
Q3: 世界上最高的现代建筑是什么？
A3: 世界上最高的现代建筑是位于阿联酋迪拜的哈利法塔（Burj Khalifa），高度为 828 米（2,717 英尺）。...
Q4: 哪些宗教场所最受游客欢迎？
A4: 这些宗教场所最受游客欢迎：

1. United States Virgin Islands的独特的宗教场所
2. Kyrgyz Republic的现代的宗教场所
3. Pitcairn Islands的美丽的宗教场所...
Q5: 有哪些著名的桥梁地标？
A5: 以下是一些著名的桥梁地标：

1. 中国的港珠澳大桥：位于中国广东省珠江口，全长55公里，是中国最著名的桥梁之一，也是世界上最长的跨海大桥之一。

2. 美国的庞洽特雷恩湖桥：位于美国田纳西州，全长10826米，是美国最著名的桥梁之一，也是世界上最长的湖桥之一。

3. 日本的东京塔桥：位于日本东京，全长386米，是东京最著名的桥梁之一，也是日本最著名的地标之一。

4. 澳大利亚的悉尼歌剧院桥：位于澳大利亚悉尼，全长798米，是悉尼最著名的桥梁之一，也是澳大利亚最著名的地标之一。

5. 英国的伦敦塔桥：位于英国伦敦，全长2025米，是伦敦最著名的桥梁之一，也是英国最著名的地标之一。

6....
--------------------------------------------------
Model-Prompt-KG: ChatGLM3-6B-思维链-KG
Total time: 22.88 seconds
Q1: 哪些国家有著名的自然景观？
A1: 根据提供的信息，我可以得出以下结论：
- Puerto Rico